In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1286583,2021-05-18,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1286584,2021-05-19,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1286585,2021-05-20,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1286586,2021-05-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
34235,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34237,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34239,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34241,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
34243,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
790483,2021-05-18,Arkansas,Arkansas,5001,2114,37.00,5000,Arkansas,AR,Arkansas,State,3017804
790485,2021-05-19,Arkansas,Arkansas,5001,2117,37.00,5000,Arkansas,AR,Arkansas,State,3017804
790487,2021-05-20,Arkansas,Arkansas,5001,2118,37.00,5000,Arkansas,AR,Arkansas,State,3017804
790489,2021-05-21,Arkansas,Arkansas,5001,2119,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1286583,2021-05-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1286584,2021-05-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1286585,2021-05-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1286586,2021-05-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-22') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
487,2021-05-22,Snohomish,Washington,38260,582.00,53061,WA,County,822083,4654.03,70.80
972,2021-05-22,Cook,Illinois,550396,10768.00,17031,IL,County,5150233,10686.82,209.08
1456,2021-05-22,Orange,California,271589,5047.00,6059,CA,County,3175692,8552.12,158.93
1939,2021-05-22,Maricopa,Arizona,547035,10010.00,4013,AZ,County,4485414,12195.86,223.17
2422,2021-05-22,Los Angeles,California,1239272,24180.00,6037,CA,County,10039107,12344.44,240.86
...,...,...,...,...,...,...,...,...,...,...,...
1285823,2021-05-22,Wheeler,Oregon,33,1.00,41069,OR,County,1332,2477.48,75.08
1286045,2021-05-22,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1286236,2021-05-22,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1286423,2021-05-22,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
487,2021-05-22,Snohomish,Washington,38260,582.00,53061,WA,County,822083,4654.03,70.80,70.80,4654.03
972,2021-05-22,Cook,Illinois,550396,10768.00,17031,IL,County,5150233,10686.82,209.08,209.08,10686.82
1456,2021-05-22,Orange,California,271589,5047.00,6059,CA,County,3175692,8552.12,158.93,158.93,8552.12
1939,2021-05-22,Maricopa,Arizona,547035,10010.00,4013,AZ,County,4485414,12195.86,223.17,223.17,12195.86
2422,2021-05-22,Los Angeles,California,1239272,24180.00,6037,CA,County,10039107,12344.44,240.86,240.86,12344.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285823,2021-05-22,Wheeler,Oregon,33,1.00,41069,OR,County,1332,2477.48,75.08,75.08,2477.48
1286045,2021-05-22,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1286236,2021-05-22,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1286423,2021-05-22,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
826608,2021-05-22,Hale,Alabama,2246,78.00,1065,AL,County,14651,15330.01,532.39,532.39,15330.01,1
899853,2021-05-22,Clarke,Alabama,3517,61.00,1025,AL,County,23622,14888.66,258.23,258.23,14888.66,2
646581,2021-05-22,Lowndes,Alabama,1413,53.00,1085,AL,County,9726,14528.07,544.93,544.93,14528.07,3
573235,2021-05-22,Etowah,Alabama,13954,357.00,1055,AL,County,102268,13644.54,349.08,349.08,13644.54,4
480936,2021-05-22,Franklin,Alabama,4266,82.00,1059,AL,County,31362,13602.45,261.46,261.46,13602.45,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218642,2021-05-22,Platte,Wyoming,636,11.00,56031,WY,County,8393,7577.74,131.06,131.06,7577.74,19
788810,2021-05-22,Converse,Wyoming,1009,17.00,56009,WY,County,13822,7299.96,122.99,122.99,7299.96,20
980975,2021-05-22,Lincoln,Wyoming,1411,12.00,56023,WY,County,19830,7115.48,60.51,60.51,7115.48,21
1092031,2021-05-22,Niobrara,Wyoming,162,2.00,56027,WY,County,2356,6876.06,84.89,84.89,6876.06,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
646581,2021-05-22,Lowndes,Alabama,1413,53.00,1085,AL,County,9726,14528.07,544.93,544.93,14528.07,3,1
826608,2021-05-22,Hale,Alabama,2246,78.00,1065,AL,County,14651,15330.01,532.39,532.39,15330.01,1,2
274788,2021-05-22,Walker,Alabama,7270,279.00,1127,AL,County,63521,11445.03,439.22,439.22,11445.03,28,3
571539,2021-05-22,Clay,Alabama,1578,57.00,1027,AL,County,13235,11922.93,430.68,430.68,11922.93,20,4
697338,2021-05-22,Greene,Alabama,927,34.00,1063,AL,County,8111,11428.92,419.18,419.18,11428.92,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695649,2021-05-22,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
956490,2021-05-22,Uinta,Wyoming,2246,13.00,56041,WY,County,20226,11104.52,64.27,64.27,11104.52,5,20
980975,2021-05-22,Lincoln,Wyoming,1411,12.00,56023,WY,County,19830,7115.48,60.51,60.51,7115.48,21,21
273498,2021-05-22,Teton,Wyoming,3776,9.00,56039,WY,County,23464,16092.74,38.36,38.36,16092.74,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4654.03,20,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4654.03,20,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4654.03,20,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4654.03,20,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.80,4654.03,20,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283573,2021-05-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1283574,2021-05-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1283575,2021-05-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1283576,2021-05-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
823180,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15330.01,2,1,1.00
823181,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15330.01,2,1,0.00
823182,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15330.01,2,1,0.00
823183,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15330.01,2,1,0.00
823184,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15330.01,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997419,2021-05-18,Crook,Wyoming,435,12.00,56011,WY,County,7584,5735.76,158.23,158.23,5775.32,6,23,1.00
997420,2021-05-19,Crook,Wyoming,435,12.00,56011,WY,County,7584,5735.76,158.23,158.23,5775.32,6,23,0.00
997421,2021-05-20,Crook,Wyoming,435,12.00,56011,WY,County,7584,5735.76,158.23,158.23,5775.32,6,23,0.00
997422,2021-05-21,Crook,Wyoming,438,12.00,56011,WY,County,7584,5775.32,158.23,158.23,5775.32,6,23,3.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
643991,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14528.07,1,3,1.00,0.00
643992,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14528.07,1,3,0.00,0.00
643993,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14528.07,1,3,0.00,0.00
643994,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14528.07,1,3,0.00,0.00
643995,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14528.07,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641871,2021-05-18,Albany,Wyoming,4451,12.00,56001,WY,County,38880,11448.05,30.86,30.86,11530.35,23,4,16.00,0.00
641872,2021-05-19,Albany,Wyoming,4457,12.00,56001,WY,County,38880,11463.48,30.86,30.86,11530.35,23,4,6.00,0.00
641873,2021-05-20,Albany,Wyoming,4472,12.00,56001,WY,County,38880,11502.06,30.86,30.86,11530.35,23,4,15.00,0.00
641874,2021-05-21,Albany,Wyoming,4483,12.00,56001,WY,County,38880,11530.35,30.86,30.86,11530.35,23,4,11.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-22') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
328076,2021-05-22,Imperial,California,28572,731.00,6025,CA,County,181215,15766.91,403.39,403.39,15766.91,1,2,21.00,0.00
2422,2021-05-22,Los Angeles,California,1239272,24180.00,6037,CA,County,10039107,12344.44,240.86,240.86,12344.44,2,5,241.00,12.00
163274,2021-05-22,San Bernardino,California,297539,4744.00,6071,CA,County,2180085,13648.05,217.61,217.61,13648.05,3,4,60.00,16.00
574883,2021-05-22,Inyo,California,1429,38.00,6027,CA,County,18039,7921.73,210.65,210.65,7921.73,4,25,0.00,0.00
72969,2021-05-22,Stanislaus,California,62543,1064.00,6099,CA,County,550660,11357.83,193.22,193.22,11357.83,5,9,32.00,0.00
33792,2021-05-22,Riverside,California,300246,4602.00,6065,CA,County,2470546,12153.02,186.27,186.27,12153.02,6,7,0.00,0.00
58045,2021-05-22,San Joaquin,California,73544,1402.00,6077,CA,County,762148,9649.57,183.95,183.95,9649.57,7,18,0.00,0.00
90486,2021-05-22,Tulare,California,50122,842.00,6107,CA,County,466195,10751.30,180.61,180.61,10751.30,8,10,0.00,0.00
32908,2021-05-22,Fresno,California,102189,1698.00,6019,CA,County,999101,10228.10,169.95,169.95,10228.10,9,14,54.00,2.00
520925,2021-05-22,Merced,California,32028,468.00,6047,CA,County,277680,11534.14,168.54,168.54,11534.14,10,8,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1218193,2021-05-22,Lassen,California,5721,24.00,6035,CA,County,30573,18712.59,78.50,78.50,18712.59,35,1,1.00,0.00
328076,2021-05-22,Imperial,California,28572,731.00,6025,CA,County,181215,15766.91,403.39,403.39,15766.91,1,2,21.00,0.00
697280,2021-05-22,Kings,California,23035,247.00,6031,CA,County,152940,15061.46,161.50,161.50,15061.46,11,3,9.00,0.00
163274,2021-05-22,San Bernardino,California,297539,4744.00,6071,CA,County,2180085,13648.05,217.61,217.61,13648.05,3,4,60.00,16.00
2422,2021-05-22,Los Angeles,California,1239272,24180.00,6037,CA,County,10039107,12344.44,240.86,240.86,12344.44,2,5,241.00,12.00
205317,2021-05-22,Kern,California,109630,1383.00,6029,CA,County,900202,12178.38,153.63,153.63,12178.38,14,6,64.00,3.00
33792,2021-05-22,Riverside,California,300246,4602.00,6065,CA,County,2470546,12153.02,186.27,186.27,12153.02,6,7,0.00,0.00
520925,2021-05-22,Merced,California,32028,468.00,6047,CA,County,277680,11534.14,168.54,168.54,11534.14,10,8,0.00,0.00
72969,2021-05-22,Stanislaus,California,62543,1064.00,6099,CA,County,550660,11357.83,193.22,193.22,11357.83,5,9,32.00,0.00
90486,2021-05-22,Tulare,California,50122,842.00,6107,CA,County,466195,10751.30,180.61,180.61,10751.30,8,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
328076,2021-05-22,Imperial,California,28572,731.00,6025,CA,County,181215,15766.91,403.39,403.39,15766.91,1,2,21.00,0.00
2422,2021-05-22,Los Angeles,California,1239272,24180.00,6037,CA,County,10039107,12344.44,240.86,240.86,12344.44,2,5,241.00,12.00
163274,2021-05-22,San Bernardino,California,297539,4744.00,6071,CA,County,2180085,13648.05,217.61,217.61,13648.05,3,4,60.00,16.00
574883,2021-05-22,Inyo,California,1429,38.00,6027,CA,County,18039,7921.73,210.65,210.65,7921.73,4,25,0.00,0.00
72969,2021-05-22,Stanislaus,California,62543,1064.00,6099,CA,County,550660,11357.83,193.22,193.22,11357.83,5,9,32.00,0.00
33792,2021-05-22,Riverside,California,300246,4602.00,6065,CA,County,2470546,12153.02,186.27,186.27,12153.02,6,7,0.00,0.00
58045,2021-05-22,San Joaquin,California,73544,1402.00,6077,CA,County,762148,9649.57,183.95,183.95,9649.57,7,18,0.00,0.00
90486,2021-05-22,Tulare,California,50122,842.00,6107,CA,County,466195,10751.30,180.61,180.61,10751.30,8,10,0.00,0.00
32908,2021-05-22,Fresno,California,102189,1698.00,6019,CA,County,999101,10228.10,169.95,169.95,10228.10,9,14,54.00,2.00
520925,2021-05-22,Merced,California,32028,468.00,6047,CA,County,277680,11534.14,168.54,168.54,11534.14,10,8,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,403.39,15766.91,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,403.39,15766.91,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,403.39,15766.91,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,403.39,15766.91,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,403.39,15766.91,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4302,05/18/21,Lassen,5716,24.00,18696.24,78.50,78.50,18712.59,35,1,2.00,0.00
4303,05/19/21,Lassen,5718,24.00,18702.78,78.50,78.50,18712.59,35,1,2.00,0.00
4304,05/20/21,Lassen,5720,24.00,18709.32,78.50,78.50,18712.59,35,1,2.00,0.00
4305,05/21/21,Lassen,5720,24.00,18709.32,78.50,78.50,18712.59,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,403.39,15766.91,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,403.39,15766.91,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,403.39,15766.91,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,403.39,15766.91,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,403.39,15766.91,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4388,05/18/21,Merced,31950,463.00,11506.05,166.74,168.54,11534.14,10,8,13.00,0.00
4389,05/19/21,Merced,31959,467.00,11509.29,168.18,168.54,11534.14,10,8,9.00,4.00
4390,05/20/21,Merced,32005,468.00,11525.86,168.54,168.54,11534.14,10,8,46.00,1.00
4391,05/21/21,Merced,32028,468.00,11534.14,168.54,168.54,11534.14,10,8,23.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)